<h1>Kubernetes</h1>

<h3>What is k8s? <h3>

Kubernetes este un sistem de orchestrare a containerelor open source dezvoltat de către Google. Acest sistem se ocupă cu partea de manageriere de containere (fie că sunt containere de tip Docker sau alte containere). Kubernetes ne ajută să manageriem aplicații care sunt create din sute sau mii de containere și ne ajută să le manageriem în environment-uri diferite precum mașini fizice (laptopuri, calculatoare), mașini virtuale sau pe cloud.

<h5>Ce tip de problemă rezolvă Kubernetes?

Care sunt task-urile unui sistem de orchestrare a containerelor?</h5>

Creșterea numărului de microservicii a dus la o creștere de utilizare a containerelor deoarece un container reprezintă ideea cea mai bună pentru o aplicație mică ce utilizează microservicii. Creșterea acestora au dus la apariția a unor aplicații create din sute sau chiar din mii de containere. Managerierea acelor containere în mai multe environment-uri  folosind script-uri sau anumite tool-uri create manual reprezintă un pas extrem de complex sau chiar imposibil. Acest scenariu a dus la nevoie de a avea tool-uri pentru orchestrarea containerelor

Ceea ce oferă aceste tool-uri: 

    - High availability = acest lucru înseamnă că aplicația nu are downtime, tot timpul putând fi accesată de către utilizator
    
    - Scalability = aplicația are o perfomanță extrem de bună, se încarcă extrem de repede iar utilizatorii au o rată de răspuns din partea aplicației foarte mare

    - Disaster Recovery = dacă o infrasctructură are o anumită problemă precum pierderea unor date sau probleme de server, infrastructura trebuie să aibă ceva mecanims de back-up pentru date pentru ca aplicația să nu piardă date

<h3> Componentele K8s</h3>

<h5>Nodes sau Pods</h5>

O să începem cu un setup basic al unui <i>worker node</i> (sau doar 'node' în kubernetes), care reprezintă un simplu server, fizic sau o mașină virtuală, iar componenta cea mai basic, sau cea mai mică unitate din kubernetes este un 'Pod'.

'Pod' este o abstracție al unui container. Un 'pod' creează un envoironment ce rulează, sau un layer peste un container, motivul fiind faptul că kubernetes dorește să abstractizeze partea de rulare a containerelor sau tehnologiile din container pentru a putea fi înclocuite dacă se dorește sau pentru faptul că nu trebuie să lucrăm cu tehnologia de Docker, ci doar pentru a interacționa cu acel layer de kubernetes.

Dacă o aplicație este cretă din mai multe containere, de cele mai multe ori în cadrul unui 'Pod' se rulează un singur container

Cum anume comunică cele 2 pod-uri în cadrul kubernetes? 

Kubernetes oferă un IP virtual, iar asta înseamnă că fiecare pod din cadrul aplicației o să primească propria adresă de IP unică. Fiecare pod poate comunica cu alt pod prin intermediul acelui IP virtual. Pod-urile în kubernetes sunt ephemerale, adică pot să 'moară' foarte ușor. În cazul acesta, dacă un pod dispare deoarece este o problemă în container și rezultă cu un crash, atunci un nou pod o să fie creat în locul lui, iar în momentul recreeri pod-ului, o nouă adresă IP virtuală o să fie assignată acelui Pod, ceea ce este incovenient în cazul în care se face comunicare între Pod-uri utilizând adresele IP. Din cauza acestei probleme, altă componentă de Kubernetes este utilizată, și anume 'Service'

<h5>Service</h5>

'Service' reprezintă o adresă IP statică (permanentă) care poate fi atașată fiecărui Pod în parte. O aplicație împărțită în 2 containere, unul cu aplicația principală și altul cu o bază de date, atunci Pod-ul cu container-ul aplicației o să aibă propriul 'Service', iar Pod-ul pentru baza de date o să aibă propriul 'Service', iar partea bună aici este faptul că durata de viață a unui 'Service' și a unui Pod nu sunt conectate, atunci chiar dacă un Pod o să 'moară', 'Service'-ul și adresa IP virtuală o să rămână.

În cazul în care dorim ca o aplicație să fie accesibilă din cadrul unui browser extern, trebuie să creem un 'Service' extern. Un 'Service' extern este un service care deschide comunicarea din surse externe, dar în mod normal nu dorim ca baza de date să aibă un service extern care să fie disponibil pentru request-urile publice, iar pentru situația de baze de date trebuie creat un service intern. 

<h5>ConfigMap și Secrets</h5>

Pod-urile după cum știm comunică între ele cu ajutorul unui Service. Aplicația cu 2 containere (unul pentru aplicația în sine, iar celălalt pentru baza de date) o să conțină un endpoint 'mongo-db-service' utilizat pentru a comunica cu baza de date, însă unde anume se confiră acest url pentru a face legătura cu baza de date? De obicei se face în fișierul de proprietăți al aplicației sau prin cadrul unei variabie de sistem externe, dar de cele mai multe ori este în imaginea aplicației (cea cu care se creează container-ul). În situația în care numele Serice-ului se modifică din 'mongo-db-service' în 'mongo-db' trebuie să modificăm acest endpoint în cadrul aplicației. Acest lucru însemană reconstrucția imaginii, push în repo pe docker-hub, pull pentru imaginea nouă în pod-ul respectiv și restartarea întregului proces. Un pic prea mulți pași pentru o simplă modificare de endpoint a bazei de date. Pentru a rezolva problema aceasta, kubernetes ne oferă partea de 'ConfigMap'

'ConfigMap' reprezintă configurația externă a aplicației. Acesta o să conțină date precum url-uri către baza de date sau alte servicii ce le utilizăm, iar în kubernetes se conectează la un pod, iar acel pod își ia datele din cadrul acelui 'ConfigMap'. Parte din cadrul datelor pentru o bază de date pot să fie reprezentate și de către un user cu parloa acestuia, date care de asemenea se pot modifica. Trecerea acestor date în cadrul 'ConfigMap' sub formă de text nu este recomandat deoarece nu este asigurată aplicația respectivă, iar pentru a rezolva această problemă, kubernetes are o altă componentă denumită 'Secrets'

<h5>Secrets</h5>

'Secrets' este asemănător cu 'ConfigMap' însă este utilizat pentru a stoca date secrete (precum user și password), datele fiind trecute în formatul base64 encoding

<h5>Volumes</h5>

Acum o să ne uităm un pic peste un alt element important din cadrul unei aplicații, și anume stocarea datelor și cum funcționează acest concept în kubernetes. Aplicația cu baza de date (explicată mai sus) are ceva date în interiorul acesteia sau produce date noi cât timp rulează aplicația. Cu conceptele de mai sus, în cazul în care Pod-ul pentru baza de date moare, datele ce au fost create cât timp a fost activ acel pod o să dispară. Modul prin care se pot păstra aceste date este prin utilizarea de Volume (precum volumele din Docker). Un volum atașează o memorie fizică de pe hard către pod-ul pentru baza de date. Acea memorie poate să fie ori pe mașina locală (pe același server node pe care rulează pod-ul) sau poate fi o memorie remote, adică în afara clusterului kubernetes.

Partea de volume este importantă deoarece kubernetes nu se ocupă și cu partea de persistența datelor, este de datoria utilizatorului să se asigure că aceste date sunt păstrate și au back-ul necesar

<h5>Deployment and Stateful Set</h5>

Din acest moment, aplicația este setată și user-ul o poate accesa dintr-un browser extern. Ce se întâmplă în situațua în care pod-ul cu aplicația de bază moare? În acest caz o să avem un moment de downtime al aplicației în care user-ul nu poate accesa aplicația respectivă (nerecomandat). În loc se ne bazăm doar pe un singur pod al aplicației sau al bazei de date, o să replicăm acestea în cadrul mai multor servere. O să avem alt Node unde o replică a aplicației o să ruleze care este de asemenea conectată la același 'Service'.

Un 'Service' are 2 funcționalități importante, oferă un IP permanent, static cu un nume constant de DNS pentru a nu fi nevoiți de fiecare dată să modificăm endpoint-ul atunci când un Pod moare, iar ca și a doua funcționalitate, Service reprezintă un 'load balancer', adică Service-ul o să prindă request-ul și o să îl trimită mai departe către orice Pod este listat ca fiind ocupat.

Pentru a crea o a doua replică a aplicației nu o să creem un pod secund, ci o să creem un blueprint al container-ului de aplicație și o să specificăm câte replici ale acelui pod dorim să ruleze, iar acel blueprint poartă denumirea de 'Deployment'. În practică nu o să lucrăm cu pod-uri, ci o să creem aceste blueprint (Deployment). După cum un pod reprezintă un layer abstractizat pentru un container, un Deployment este un alt layer abstractizat peste pods.

<h5>Stateful Set</h5>

În cazul acesta, dacă pod-ul aplicației o să dea crash, atunci Service-ul o să facă forward la request către alt pod care rulează pentru ca aplicație să fie accesibilă pentru utilizator. Ce se poate face pentru pod-ul care se ocupă cu partea de baze de date? Dacă acel pod moare, atunci aplicația din nou nu o să fie accesibilă de către utilizator. Ideea este că nu se pot replica baze de date utilizând conceptul de 'Deployment', iar motivul pentru acesta este faptul că o bază de date are un anume 'state', care reprezintă datele din interiorul bazei respective. În cazul în care avem clone ale aceleași baze de date, toate trebuie să acceseze aceleași date, lucru care necesită un mecanism pentru a verifica care anume pod rulează query-uri de scriere în baza de date sau ce pod-uri preia date din baza de date. Acest mecanism este oferit de către kubernetes prin 'Stateful Set'.

Acest concept este utilizat în special pentru aplicații ce utilizează și baze de date. Precum 'Deployment', 'Stateful Set' se ocupă cu partea de replicare a pod-urilor și se asigură că partea de scriere și citire din baza de date sunt sincronizate pentru a nu oferi inconsistențe în baza de date. Totuși, partea de deploy pentru 'Stateful Set' în clustere de Kubermetes nu este tocmai ușoară

<h3>K8s Architecture</h3>

<h5>Node Processes</h5>

Una dintre componentele principale din kubernetes este reprezentată de 'worker service' ('nodes'). Fiecare 'Node' o să aibă mai multe pod-uri care rulează în cadrul acelui 'Node'. Modul prin care kubernetes realizează acest lucru este prin intermediul a 3 procese care trebuie instalate pentru fiecare 'Node' care este utilizat pentru a programa și a manageria acele părți. 'Nodes' reprezintă partea dintr-un cluster kubernetes care practic lucrează (de aici și denumirea de 'Worker Nodes').

Primul proces care trebuie să ruleze pentru fiecare 'Node' este procesul de container (container runtime, docker de exemplu). Deoarece pod-urile aplicației au containere care rulează în interior, un container runtime trebie să fie instalat în cadrul fiecărui Node, dar procesul care se ocupă cu partea de programare pentru pod-uri și containerele pentru care rulează acele pod-uri este 'Kubelet'

'Kubelet' este un proces de ține de kubernetes care are interfațe pentru atât partea de container runtime cât și pentru Node-ul în sine. Kubelet este responsabil să ia configurația respectivă și să ruleze sau să pornească un anume Pod (cu un container respectiv) și să assigneze resurse din acel Node pentru container (CPU, RAM, etc)

De cele mai multe ori un cluster kubernetes este creat din mai multe Nodes, fiecare dintre ele având procesele de container runtime (Docker) și Kubelet. Modul prin care node-urile comunică între ele în cadrul unui cluster este prin 'Services', acesta preia request-ul făcut către un anumit Pod și în redirecționează către un anume Pod.

Al treilea proces care se ocupă cu partea de redirecționare a unui request către un anume Pod poartă denumirea de 'Kube Proxy', fiind necesar și acesta să fie instalat pe fiecare Node în parte. 'Kube proxy' are un mecanism inteligent de redirecționare care se asigură că această comunicare se face într-un mod performat prin care redirecționeazî request-ul către pod-uri din același Node

<h5>

Cum se interacționează cu acest cluster kubernetes?

Cum se decide pe ce nod sau pod aplicația o să fie programată?

Dacă un pod moare, ce proces se ocupă cu partea de monitorizare și repornire? 

Când se adaugă un nou server cum se alătură cluster-ului pentru a deveni un nou Node? </h5>

Pentru a răspunde la întrebările de mai sus, trebuie să aruncăm o privire peste 'Master Node'

<h5>Master Node</h5>

Un 'Master Node' are procese complet diferite care rulează în interiorul lor. Există 4 procese care rulează pe fiecare 'Master Node' ce controlează 'cluster state' și 'Worker Node'.

Primul proces este 'API Server'. Atunci când ca user dorești să faci deploy la o nouă aplicație într-un cluster de kubernetes o să interacționezi cu un 'API Server' prin intermediul unui 'client' (interfață grafică, linie de comandă (kubelet), Kubernetes API)

<h5>Scheduler</h5>

Un alt proces este cel 'Scheduler'. Atunci când se face un request la serverul API pentru a programa un anumit Pod, după ce acest request este validat acesta este trimis către 'Scheduler' pentru a porni un Pod al aplicației în cadrul unuia dintre nod-uri (worker node). Pod-ul o să fie pornit pe un Node care utilizează cele mai puține resurse, nu se pornește un Pod random pe un anumit Node, acest sistem are un mecanism inteligent de programare. Acest 'Scheduler' doar decide pe ce Node acest Pod o să fie pornit, procesul care pornește acel Pod este 'Kubelet'.

<h5>Controller Manager</h5>

Acest proces detectează schimbări în cluster, precum crash a unui anumit Pod dintr-un anumit Worker Node. Când detectează că un Pod a murit, face un request la 'Scheduler' pentru a reporni acele Pod-uri care au murit

<h5>etcd</h5>

Acesta este o stocare de tipul key-value pentru partea de 'cluster state'. Putem să ne imagină ca fiind creierul clusterului. Orice schimbare ce se petrece în cadrul unui cluster (reprogramare unui Pod, dacă a murit un Pod) sunt salvate sau updatate în acest sistem de stocare key-value. Motivul pentru care aceste 'etcd' este considerat ca fiind creierul clusterului este pentru faptul că toate procesele de mai sus funcționează datorită datelor din cadrul clusterului (date stocate în etcd)

<h3> Cluster example</h3>

Într-un cluster kubernetes mic o să avem cel mai probabil 2 Master Nodes și 3 Worker Nodes. Deși Master Nodes sunt mai importante, acestea consumă mai puține resurse deoarece toată treaba este realizată de către WOrker Nodes. Un Master Nodes. Fiecare Node o să aibă o proprie mașină (fie fizică sau virtuală). Dacă se dorește să se testeze ceva local, setarea unui cluster este extrem de dificilă, poate chiar imposibilă deoarece este nevoie de resurse mari (CPU, RAM). Pentru acest pas de testare locală se poate uiliza 'Minikube'

<h5>Minikube</h5>

Minikube este un Node din Cluster unde procesele din Master Node și Worker Node funcționează pe același Node. Modul prin care acestea rulează este cu ajutorul unui VM. Minikube este un cluster de 1 Node din Kubernetes care rulează pe un Vortual Box. Din moment ce avem acest Node pe mașina locală în cluster, avem nevoie de un mod de a interacționa cu acest Node. Această interacțiune se face cu 'kubectl'

<h5>kubectl</h5>

'kubectl' este o linie de comandă pentru clusterele kubernetes. Cum se preciza, atât procese de la Worker Node și Master Node rulează pe același Node cu 'minikube', iar unul dintre procesele ce rulează din Master Node este cel de API Server. Pentru a realiza orice modificări în clusterul de Kubernetes întâi trebuie să interacționă cu API-ul de la server. Interacțiunea acesta se poate face fie printr-o interfață grafică, fie prin API-ul de Kubernetes, fie prin linia de comandă (care este kubetcl). 'kubectl' reprezintă cel mai puternic mod de interacțiune cu API Server deoarece prin linia de comandă se pot realiza toate comenzile din kubernetes.

Pentru a putea crea acest cluster local trebuie să instalăm următoarele tehnologii:

    1. minikube

    2. kubectl

    3. VirtualBox

După ce am instalat aceste tehnologii putem să creem un cluster de kubernetes. Pentru a crea acest cluster o să se utilizeze 'minikube'. Comanda pentru a crea un cluster este 'start'.

In [ ]:
minikube start

<img src='./ScreenShots/minikube_start.png'>

După ce am pornit cluster-ul local, putem verifica Node-urile din acest cluster. Pentru a verifica nodurile se va utiliza 'kubectl'. Acestei comenzi i se va adăuga 'get nodes'

In [ ]:
kubectl get nodes

<img src='./ScreenShots/kubectl_get_nodes.png'>

'minikube' se va utiliza doar pentru a crea, a porni, a opri sau a șterge un cluster local. Pentru restul acțiunilor ce țin de cluster o să ne folosim de 'kubectl'. Linia de comandă 'kubectl' se poate folosi și pentru clustere de producție găsite pe un cloud sau pe mașini virtuale, nu este specific doar pentru 'minikube'

<h3>Basic kubelet commands</h3>

În acest moment avem un cluster format cu 'minikube' și linia de comandă 'kubectl' instalată. Din moment ce clusterul este setat, o să ne folosim de 'kubectl' pentru a face orice modificări în interiorul clusterului. Pentru a vedea Nodes care sunt prezente în cadrul clusterului am observat că se utilizează comanda 'kubectl get nodes'. În ceea ce privește Pods, pentru a vedea toate Pods se utilizează comanda 'kubectl get pod'

In [ ]:
kubectl get pod

<img src ='./ScreenShots/kubectl_get_pod.png'>

În acest moment se poate observa că în cadrul acestui cluster nu există niciun Pod care rulează.

Pentru parte de Service din cadrul unui Node se va utiliza comanda 'kubectl get service'

<img src='./ScreenShots/kubectl_get_service.png'>

Prin comanda de mai sus ni se afișează datele referitoare la partea de Service din cadrul Node. În aceste date se găsește adresa IP a clusterului respectiv.

Din moment ce nu avem niciun Pod în cadrul Clusterului, o să începem să creem. Pentru a crea componente în cadrul clusterului o să se utilizeze comanda 'create' din 'kubectl'. Dacă se utilizează --help pentru 'kubectl create' se va observa că nu există o comandă ca să creeze un Pod.

<img src='./ScreenShots/kubectl_create_help.png'>

Nu există opțiunea de a crea un Pod deoarece modul în care kubernetes funcționează, un Pod este cea mai mică unitate din cadrul clusterului , dar în practică nu o să creem Pods sau nu o să lucrăm cu Pods, există un layer abstract peste Pods care poartă denumirea de Deployment (cum este specificat mai sus). O să creem un 'deployment', iar acesta o să creeze aceste Pods în spate. Metoda de folosire a comenzii este următoarea:

In [ ]:
kubectl create deployment NAME --image=image [--dry-run] [options]

Pentru a crea un deployment trebuie să îi oferim un nume acestuia și o imagine deoarece un Pod este creat pe baza unei imagini (iar container-ul este creat pe baza acestei imagini). În continuare o să creem un deployment după imaginea de 'nginx'

In [ ]:
kubectl create deployment nginx-deployment --image=nginx

<img src='./ScreenShots/kubectl_create_deployment_nginx.png'>

Cu această comandă o să se creeze un deployment utilizând imaginea de nginx. Comanda descarcă de pe DockerHub imaginea de nginx și creează containere pe baza acestei imagini. Output-ul comenzii ne indică faptul că acest deployment a fost creat. Cu comanda 'get deployment' (din cadrul 'kubectl') putem să afișăm informații referitoare la acest deployment.

In [ ]:
kubectl get deployment

<img src='./ScreenShots/kubectl_get_deployment.png'>

După ce am creat acest deployment putem verifica în acest moment dacă s-a creat ceva Pod în cadrul acestui deployment. Pentru acesta o să utilizăm comanda 'kubectl get pod'

In [ ]:
kubectl get pod

<img src='./ScreenShots/kubectl_get_pod_2.png'>

După ce am creat acest deployment putem observa că în acest moment avem un Pod care rulează. Numele Pod-ului este prefixat de către numele deployment-ului după care se mai adaugă și un hash. Modul în care funcționează, în momentul în care am creat acest 'nginx-deployment', acest deployment are toate informațiile necesare (un blueprint) pentru a crea acest Pod. Comanda de mai sus este cea mai minimalistică metodă de a crea un deployment, îi oferim doar un nume și o imagine pe baza căreia să creeze Pods. Între deployment și Pod există alt layer care este manageriat automat de către deployment-ul kubernetes care poartă denumirea de 'replicaset'. Datele despre acest layer pot fi opțiunute utilizând comanda 'kubectl get replicaset'

In [ ]:
kubectl get replicaset

<img src='./ScreenShots/kubectl_get_replicaset.png'>

Acest 'replicaset' are și el un nume specific și este creat din numele deployment-ului ca și prefix, plus o serie hash. Dacă se poate observa, numele Pod-ului este creat din 2 prefixe (numele deployment-ului, hash-ul 'replicaset') și la acestea se mai adaugă încă un hash.

'replicaset' se ocupă cu partea de manageriere a replicilor unui Pod. Ca și utilizator în practică nu o să avem să ne creem un 'replicaset' sau să ștergem, să modificăm o astfel de componentă, o să ne ocupăm doar de partea de deployment, în partea de deployment se poate modifica blueprint-ul acestuia și există posibilitatea să specificăm câte replici să fie în cadrul Node

Toate modificările care trebuie să se realizeze în partea de deployment o să fie realizate utilizând partea de linie de comandă din kubernetes ('kubectl'). Pentru a modifica imaginea cu care a fost creat deployment-ul 'nginx-deployment', 'kubectl' are la dispoziție opțiunea de 'edit deployment', iar pentru acesta trebuie să specificăm numele deployment-ului pe care dorim să îl modificăm

In [ ]:
kubectl edit nginx-deployment

Comanda de mai sus o să ne deschisă în terminal fișierul de configurație al deployment-ului, iar în cadrul acestuia o să navigăm la secțiunea unde găsim imaginea pe baza căreia este creat Pod și o să modificăm din 'nginx' în 'nginx:1.16'

<img src='./ScreenShots/kubectl_edit_deployment.png'>

După ce am modificat acest fișier o să salvăm fișierul și o să ieșim din el. În terminal o să se afișeze mesajul cum că acest deployment a fost editat, dacă se rulează comanda 'kubectl get pod' se va observa că Pod-ul vechi o să fie închis și o să se creeze alt Pod în acest moment.

<img src='./ScreenShots/edited_deployment.png'>

În kubernetes există o listă a abstractizării, iar aceasta arată așa:

    1. deployment

    2. replicaset

    3. pod

    4. container

Tot ce se găsește mai jos de partea de 'deployment' o să fie manageriat de kubernetes, de aceea dacă am modificat deployment-ul anterior s-au modificat și Pods. Deoarece 'replicaset' se găsește mai jos de deployment și aceasta o să fie modificată. O nouă 'replicaset' va fi creată, iar în cea anterioară se va observa că nu mai are niciun Pod care rulează.

<img src='./ScreenShots/kubectl_get_replicaset2.png'>

<h3>Debugging Pods</h3>

O altă comandă utilă pentru kubernetes este cea de 'logs'. Comanda respectivă afișează ce se loghează de către aplicația care rulează în cadrul Pod. Sintaxa comenzii este următoarea: kubectl logs [pod name]. Deoarece numele unui Pod este greu de reținut este recomandat să se afișeze inițial Pods (kubectl get pod) și să se ia cu copy paste numele Pod la care dorim să facem referire.

In [ ]:
kubectl get pod

In [ ]:
kubectl logs nginx-deployment-6cdd747bb4-6sgbg

<img src='./ScreenShots/kubectl_logs.png'>

Pentru comanda de mai sus nu există niciun output deoarece aplicația nu a scris niciun log (așa este setată). În continuare o să creem un deployment utilizând imaginea de MongoDB (mongo) care o să scrie anumie log-uri în momentul în care se creează un container pe baza imaginii.

In [ ]:
kubectl create deployment mongo-deployment --image=mongo

<img src='./ScreenShots/kubectl_logs_mongo.png'>

Din moment ce Pod-ul rulează în cadrul container-ului putem să afișăm log-uril ce au fost scrise de către aplicație. Log-urile respective pot ajuta foarte mult la partea de debugging în cazul în care există ceva erori în container și nu rulează în modul în care acesta ar trebui.

<img src='./ScreenShots/kubectl_logs_mongo2.png'>

O altă comandă utilă pentru kubernetes este comanda 'exec' (precum cea din Docker). Această comandă ne permite sî intrăm în cadrul interiorul containerului ce se rulează sub formă de terminal.

In [ ]:
kubectl exec -it mongo-deployment-686c5687fb-x7w8z -- bash

Argumentul '-it' specifică să se deschidă un terminal în modul interactiv, iar partea de la final ('-- bash') reprezintă comanda care să fie executată în paralel cu comanda default ce se execută în imaginea docker. Comanda este utilă pentru partea de debuggind, prin această comandă se poate intra în terminal în container-ul respectiv, iar de acolo se pot realiza anumite modificări dacă este necesar.

<img src='./ScreenShots/kubectl_exec.png'>

La fel cum cu ajutorul 'kubectl' se creează un deployment (care creează automat 'replicaset' și Pods), tot utilizând linia de comandă 'kubectl' se pot șterge aceste deployment. Pentru a șterge un deployment se va utiliza comanda 'delete deployment' urmată de numele deployment-ului pe care dorim să îl ștergem. Comanda de 'delete deployment' o să șteargă și tot ce se găsește mai jos de deployment, și anume replicaset și Pods.

In [ ]:
kubectl detele deployment mongo-deployment

<img src='./ScreenShots/kubectl_delete.png'>

Se poate observa că din moment ce s-a șters deployment-ul 'mongo-deployment', atât 'replicaset' și Pods ce au ținut de acest deployment au fost șterse.

Inițial, când am creat aceste deployment-uri am specificat doar elementele necesare pentru a se crea un deployment, și anume partea de name a deploymentului repsectiv și imaginea docker pe baza căreia să se creeze Pods. De foarte multe ori, în momentul în care se creează in deployment pentru acesta trebuie specificate mai multe opțiuni. Pentru a nu scrie o linie de comandă extrem de lungă și de complicată, de cele mai multe pri să utilăm partea de 'configuration file' din kubernetes.

Componentele care sunt creat, imaginea pe care o va utiliza deployment-ul și oricare alte opțiuni, toate o să fie trecute în cadrul acestui fișier de configurare, iar noi doar o să îi spunem lui 'kubectl' să execute acel fișier de configurare. Modul prin care se realizează acest pas este utilizând comanda 'kubectl apply'. Comanda respectivă ia un fișier de configurație ca și parametru și execută tot ce este trecut în acel fișier.

In [ ]:
kubectl apply -f [file name]

Fișierul respectiv este de tipul .yaml și în el sunt prezente toate informațiile necesare pentru a crea și rula un deployment. O să creem un fișier nginx-deployment.yaml unde o să trecem informațiile necesare pentru a crea și rula un deployment.

<img src='./ScreenShots/nginx_yaml.png'>

Fișierul respectiv este cel din imaginea de sus. 'kind: Deployment' îi specifică lui kubernetes să creeze un Deployment, iar pentru acest deployment, în 'metadata' se trece numele deployment-ului pe care dorim să îl aibă ('name: nginx-deployment'). Cu linia 'replicas: 1' se specifică câte replici ale unui Pod să se creeze în acest Node. Ce este trecut în partea de 'template' (și mai jos de acesta) reprezintă blueprint-ul pentru Pod-urile care o să fie create în cadrul unui Worker Node. Pentru a crea containere se va utiliza imaginea de nginx care o să fie luată de pe Docker Hub.

Dacă se rulează comanda 'kubectl apply -f nginx-deployment.yaml', acest deployment ar trebuie să se creeze automat cu un Pod în cadrul acestui deployment.

In [ ]:
kubectl apply -f nginx-deployment.yaml

<img src='./ScreenShots/kubectl_apply.png'>

Se poate observa că acest deployment a fost creat și rulează corect. Dacă se dorește să se modifice ceva anume în cadrul acestui deployment, în momentul acesta se poate modifica în cadrul fițierului local. În continuare o să modificăm fișierul de configurație ca acesta să aibă 2 replici.

<img src='./ScreenShots/nginx_yaml2.png'>

După ce am realizat acestă modificare în cadrul fișierului de configurație se poate rula din nou comanda 'kubectl apply -f nginx-deployment.yaml'

In [ ]:
kubectl apply -f nginx-deployment.yaml

Kubernetes este destul de deștept încât să știe dacă să creeze sau doar să facă update la un deployment. În prima parte, când s-a rulat comanda 'kubectl apply -f nginx-deployment.yaml' kubernetes a creat acest deployment, iar după ce am modificat în cadrul fișierului de configurație și am rulat din nou comanda, acest deployment a fost updatat. 

<img src='./ScreenShots/kubectl_apply2.png'>

Și din output-ul comenzii se poate observa că doar s-a făcut update la deployment, nu a fost creat un deployment nou (deployment.apps/nginx-deployment configured).

<h3>YAML configuration file</h3>

Fiecare fișier de configurație din Kubernetes are 3 părți.

Prima parte este reprezentată de metadata (informațiile, datele despre deployment, service, sau ce se creează). A doua parte este reprezentată de către specificații  , specificații pentru componenta care se dorește să fie creată. Atributele din partea de specificații o să fie diferite în funcție de tipul (kind) de componentă pe care îl creem (deployment, service).

Cea de a 3-a partea din cadrul fișierului de configurație este reprezentat de un status. Acest 'status' este adăugat și manageriat automat de către kubernetes. Kubernetes face o comparație între fișierul de configurație yaml (ceea ce se dorește să existe în cluster) și ceea ce anume există în cluster. De exemplu dacă în fișier se specifică că se dorește să fie 2 replici, dar numai 1 este activă kubernetes depistează această diferență între ce anume există (doar 1 replică) și ce se dorește să existe (2 replici) și încearcă remedierea problemei cât mai rapid. Această a 3-a parte de 'status' își ia informațiile necesare din partea de 'etcd'.

După cum se preciza, un deployment se ocupă cu tot ce ține de Pods, acestea fiind configurate în cadrul fișierului yaml. Unde anume se găsește această configurare? Prima parte de 'spec' din cadrul fișierului are o secțiune de 'template'. Această secțiune de template este împărțită la rândui în 'metadata' și 'spec'. Secțiunea de 'template' reprezintă configurația unui Pod. Practic este un fișier de configurație în interiorul altui sistem de configurație. 

Modul prin care conexiunea se realizează este prin utilizarea de 'labels' și 'selectors'. Partea de 'metadata' (cea pentru deployment, nu cea pentru Pod) conține partea de 'labels', iar în cadrul secțiunii de 'specs' se găsește partea de 'selectors'

În secțiunea de 'metadata' se oferă un set de valori de tipul key-value (app: nginx), iar acest 'label' o să se 'lipească' de componenta unde anume l-am setat. Cu linia de cod 'matchLabel' match-uim un anumit Pod la deployment, iar în acest fel se știe ce Pods ține de acest deployment.

Prin partea de 'labels' din template-ul pentru Pods match-uim un Pod la un deployment. Partea de metadata din deployment are și ea o secțiune de 'labels' (secțiune care este diferită de 'labels' din template), iar prin aceasta se va face conexiunea cu un 'Service' (unde în service va exista linia de cod 'selector: app: nginx')

Următorul lucru care trebuie configurat în cadrul fișierului de configurație (valabil pentru Pods și Service) îl reprezintă port-urile. Un Service are o secțiune separată pentru porturi, iar un container din cadrul unui Pod trebuie să ruleze pe un anumit port. Partea de service are un port unde Service-ul în sine este accesibil (poartă denumirea de 'port'), dar service-ul trebuie să știe către ce Pod trebuie să trimită mai departe request-ul, dar și ce port a deschis acel Pod (poartă denumirea de 'targetPort'). Acest 'targetPort' trebuie să fie la fel ca și port-ul deschis în cadrul container-ului din Pod (un port din container se deschide prin comanda 'containerPort' și trebuie să aibă aceeași valoare ca 'targetPort')

Pentru a înțelege mai bine, o să afișăm 2 imagini, una cu deployment-ul de nginx, iar alta cu service-ul de nginx

<img src='./ScreenShots/nginx-deployment.png'>

<img src=./ScreenShots/nginx-service.png>

Se poate observa că partea de 'containerPort.yaml' din fișierul 'nginx-deployment' are aceeași valoare ca și 'targetPort' din fișierul 'nginx-sercice.yaml'

Majoritate atributelor prezente în cadrul celor 2 fișiere reprezintă atributele care sunt necesare pentru a crea un deployment valid. Din moment ce avem aceste fișiere de tip yaml, unul cu un Service, iar altul cu un deployment, o să ne folosim de aceste fișiere pentru a crea componente.

In [ ]:
kubectl apply -f nginx-deployment.yaml

In [ ]:
kubectl apply -f nginx-service.yaml

In [ ]:
kubectl get pod

<img src='./ScreenShots/kubectl_apply_service.png'>

După utilzarea comenzilor putem observa că atât pod-urile, deploymnet-ul și service-ul au fost create și rulează. Dacă se va rula comanda de a ne afișa service-ul, noul service creat va fi afișat în listă

<img src='./ScreenShots/kubectl_get_service2.png'>

În lista de mai sus se poate observa că există 2 service, însă cel de kubernetes este creat automat. Se poate observa că acest service rulează pe portul 80, după cum a fost specificat în cadrul fișierul 'nginx-service.yaml'. Cum anume se poate valida că acest service are pod-urile necesare și că face forward la request către pod-urile care trebuie. Pentru a verifica acest lucru se va utiliza comanda 'kubectl describe service' și numele serviciului.

In [ ]:
kubectl describe service nginx-service

<img src='./ScreenShots/kubectl_describe_service.png'>

În output-ul acestei comenzi avem informații precum partea de 'selector', IP-ul clusterului, port-ul pe care listează clusterul (Port: 80/TCP) și targetPort (targetPort: 8080) precum au fost specificate în fișierul 'nginx-service.yaml' La partea de 'Endpoints' sunt trecute IP-urile și port-urile pod-urilor pentru care acest service trebuie să facă forward la request-urile pe care le primește. Cu comanda de 'kubectl get pod' partea de IP a pod-ului nu este afișată, însă există un argument pentru acestă comandă care îi spune lui kubernetes să ne afișeze mai multe date despre pod-uri. Acest flag poartă denumirea de '-o' are provine de la output și specificăm că dorim să avem un output 'wide'

In [ ]:
kubectl get pod -o wide

<img src='./ScreenShots/kubectl_get_pod_wide.png'>

Cu acestă comandă se poate observa că pod-urile respective au aceleași IP-uri precum sunt specificate în partea de 'Endpoints' de la descrierea serice-ului, ceea ce înseamnă că aceste pod-uri fac parte din acest service.

<h3>Complete Demo Project</h3>

În acest tutorial o să facem deploy la 2 aplicații, mongidb și mongoexpress. Demostrează foarte bine un setup tipic de aplicație web împreună cu baza ei de date. Pentru acest proiect o să creem următoarele componente:

    1. 2 Deployments

    2. 2 Services

    3. 1 ConfigMap

    4. 1 Secret

O să creem un Service intern, prin care doar componentele din cadrul clusterului o să poată comunica cu acel service, pe urmă o să creem un deployment de mongoexpress împreună cu un url de mongodb (pentru a se conecta la baza de date) împreună cu partea de credințiale. Aceste date o să fie trecute ca și environmental variables în cadrul deployment-ul. Un ConfigMap o să fie creat pentru URL-ul de bază de date și un Secret pentru partea de credințiale. La acestea o să face referire în cadrul deployment-ului.

După ce am creat această parte, o să avem nevoie ca mongoexpress să fie accesibil dintr-un browser, prin urmare o să avem un service extern care o să permită requesturilor externe să realizeze conexiune cu pod-ul.

Am creat un director separat în cadrul mașinii pentru a crea acest proiect. Primul pas în cadrul proiectului este de a crea un deployment pentru partea de mongodb

<img src='./ScreenShots/Demo/mongodb_deployment.png'>

Pentru început așa o să arate fișierul 'mongodb-deployment.yaml'. Este un fișier ce conține momenta doar strictul necesar pentru a crea un deployment. În continuare o să navigăm către Docker Hub pentru a verifica imaginea 'mongo', pentru a vedea dacă avem nevoie de date adiționale pentru a rula un container pe baza imaginii.

Ce anume trebuie să căutăm în cadrul imaginii de mongo? Ce porturi deschide container-ul respectiv și dacă se utilizează ceva envoironmental variables pentru acestă imagine. Partea de porturi se găsește în secțiunea de 'Connect to MongoDB from another Docker container' unde se precizează că se utilizează portul 27017, iar la secțiunea de 'Environment Variables' se găsesc informațiile despre environmental variables

<img src='./ScreenShots/Demo/mongodb_port.png'>

<img src='./ScreenShots/Demo/mongodb_envs.png'>

La secțiunea de 'Environment Variables' se poate observa că se utilizează 2 variabile de sistem, și anume:

    MONGO_INITDB_ROOT_USERNAME

    MONGO_INITDB_ROOT_PASSWORD

Pentru început o să specificăm port-ul ce trebuie deschis în cadrul container-ului în fișierul de configurație yaml

In [ ]:
ports:
  - containerPort: 27017

În continuare trebuie să specificăm 2 variabile de sistem. Acestea se introduc într-un fișier de configurație cu 'env'. Pentru fiecare variabilă trebie specificat nmele acesteia (- name) și o valoare (pe care momentan o să o lăsăm fără valoare)

In [ ]:
env:
  - name: MONGO_INITDB_ROOT_USERNAME
  value:
  - name: MONGO_INITDB_ROOT_PASSWORD
  value:

<img src='./ScreenShots/Demo/mongodb_deployment2.png'>

După ce se completează și la secțiunea de 'value' din partea de 'env' totul este setat pentru partea de deployment de la mongodb. Acest fișier este unul care o să fie pus în cadrul unui repository, prin urmare nu este recomanda să se treacă credințiale sub formă de text în cadrul acestui fișier. În continuare o să creem o componentă de 'Secret' cu ajutorul căreia o să facem referințe la aceste credințiale. Această componentă de 'Secret' face parte din kubernetes, nimeni nu o să aibă acces la ea dintr-un repository

 Configurația unei componente de tipul 'Secret' arată în felul următor:

<img src='./ScreenShots/Demo/mongodb_secret.png'>

În cadrul acestui fișier avem un 'kind', care este de tipul 'Secret' de această dată (în loc de Deployment), avem o secțiune de 'metadata' (care reprezintă numele acestei componente 'Secret'). Partea de 'type: Opaque' este setată default, iar acest tip reprezintă cel mai basic tip de date secrete de tipul key-value. În partea de 'data' sunt trecute perechile key-value care sunt specificate de către developer (username sau password se pot modifica în orice alte denumiri).

Valorile care sunt trecute în secțiunea de 'values' (adică după semnul două puncte) nu sunt trecute în text. La valoare, aceasta trebuie să fie de tipul 'base64-encoded'. Cel mai simplu mod de o obține o valoare base64 encoded este de a utiliza terminalul.

In [ ]:
echo -n 'username' | base64

Comanda de mai sus afișează valoare în formatul 'base64' pentru 'username'. Outputul acestei comenzi poate fi trecut ca și value în cadrul fișierului de configurație. La fel se poate proceda și pentru partea de 'password'. Valorile ce sunt trecute între glimile reprezintă valoare în text, prin urmare acesta este definită de către programator în funcție de username-ul și parola ce o are.

<img src='./ScreenShots/Demo/echo_base64.png'>

<img src='./ScreenShots/Demo/mongodb_secret2.png'>

Până în acest moment doar am scris fișiere, nu am creat nimic concret în cadrul clusterului, aceasta a fost doar muncă de pregătire. Dacă dorim să putem face referire în partea de deployment la valorile din secret, atunci componenta de secret trebuie creată înainte de componenta de deployment. Dacă se creează un deployment care face referire la o componentă de secret care nu există, atunci o să primim o eroare. În continuare o să creem componenta de secret

In [ ]:
kubectl apply -f mongodb-secret.yaml

<img src='./ScreenShots/Demo/kubectl_get_secret.png'>

Odată cu rulearea comenzii pentru fișierul 'mongodb-secret.yaml' se poate observa că această componentă de secret a fost creată cu succes. Din moment ce avem secret-ul creat, putem să facem referire la acesta în cadrul fișierului 'mongodb-deployment.yaml'. Pentru a face referință într-un fișier de deployment la un set de date key-value dintr-un secret se va utiliza sintaxa următoare:

In [ ]:
valueFrom:
  secretKeyRef:
    name: <Name of secret>
    key: <Key of data to be refrenced>

'name' reprezintă numele secret-ului (mongodb-secret în cazul de față). Aceast nume este cel stabilit în secțiunea de metadata, pentru atributul 'name' ('name: mongodb-secret'). Partea de 'key' reprezintă key-ul din secțiunea de 'data' din fișierul de secret. Dacă în fișierul de secret avem perechea 'username_key: username_value', pentru a prelua valoarea (adică 'username_value') trebuie să trecem la 'key' cheia pentru care dorim să preluăm valoarea, adică 'username_key'. Sintaxa de mai sus nu trebuie ținută minte, doar să se știe ca și idee cum se face referire la un sercret în cadrul deployment-ului. Sintaxa respectivă se poate căuta și pe internet

<img src='./ScreenShots/Demo/mongodb-deployment3.png'>

Deployment-ul final 'mongodb-deployment.yaml' o să arate așa (precum în imaginea de sus). În acest moment avem referințele către credințiale, prin urmare acestea nu o să fie trecute ca și text în fișierul de deployment, ceea ce este bine pentru partea de securitate. Din acest moment putem să aplicăm acest fișier de deployment utilizând 'kubectl apply -f'

In [ ]:
kubectl apply -f mongodb-deployment.yaml

<img src='./ScreenShots/Demo/kubectl_apply_get_deployment.png'>

Acum avem partea de mongodb cu 3 replici care rulează. Pasul următor este acela de a crea un server intern pentru ca alte componente din cadrul clusterului (alte pods) să poată să interacționeze cu partea de mongodb. Putem fie să creem un fișier separat de tipul yaml pentru service, sau se poate include în cadrul fișierului de deployment. În yaml se pot pune mai multe documente într-un singur fișier. Dacă se adaugă trei linii (---) aceasta reprezintă o sintaxă pentru separarea documentelor în yaml. O să creem și partea de service tot în cadrul documentului de deployment deoarece aceste se leagă unul de altul.

<img src='./ScreenShots/Demo/mongodb-deployment-secret.png'>

Un fișier de tipul Service este destul de simplu, partea de 'kind' îi spune lui kubernetes că dorim să creem un 'Service'. În secțiunea de metadata după cum bine știm îi atribuim un nume acestui service. Partea importantă apare la secțiune de 'selector', deoarece acest service trebuie să știe la pods să se conecteze, iar modul de a face aceasta este prin utilizarea unui selector. Partea de porturi iar este foarte importantă pentru un service. Partea de 'port' din 'ports' reprezintă port-ul pentru care listează service-ul, iar secțiunea de 'targetPort' reprezintă portul pe care trebuie să listeze container-ul. 'targetPort' din Service și 'containerPort' din Deployment trebuie să aibă aceeași valoare. Partea de 'port' și 'targetPort' din cadrul Service pot să aibă valori diferite.

Cam așa arată componenta de Service pentru mongodb, din acest moment putem să creem această componentă.

In [ ]:
kubectl apply -f mongodb-deployment.yaml

<img src='./ScreenShots/Demo/kubectl_apply_service.png'>

Deși am creat atât componenta de deployment și service în același fișier, kubernetes este destul de deștept pentru a le diferenția. Partea de 'Deployment' a rămas modificată în cadrul fișierului, prin urmare, de aceea în output se afișează faptul că componenta Deployment a rămas neschimbată și că s-a creat o nouă componentă de service.

Pasul următor este de a crea partea ce ține de mongo-express (deployment, service, configmap). O să începem cu partea de deployment pentru mongo-express. Pentru aceasta o să creem un nou fișier de tip yaml. La fel ca și în cazul imaginii de mongo, o să navigăm pe Docker Hub și o să căutăm informații cu privire la modul în care se rulează imaginea respectivă (porturi deschise și variabile de sistem).

 <img src='./ScreenShots/Demo/mongo-express_docker_hub.png'>

La descrierea imaginii se poate observa că se utilizează port-ul 8081, iar în ceea ce privește partea de variabile de sistem, imaginea respectivă are ami multe variabile. Deoarece dorim să ne conectăm la o bază de date de tipul mongodb, ne interesează partea de admin username și password pentru mongodb (ME_CONFIG_MONGODB_ADMINUSERNAME și ME_CONFIG_MONGODB_ADMINPASSWORD) și serverul la care să ne conectăm (ME_CONFIG_MONGODB_SERVER).

Partea de credințiale o să fie cele care sunt trecute în Secret-ul de mongodb, prin urmare putem să le referențiem aici precum am făcut și în cadrul deployment-ului de mongodb.

Partea de 'ME_CONFIG_MONGODB_SERVER', din moment ce este tot configurație de tip extern putem să utilizăm partea de 'value' și să îi atribuim adresa serverului de mongodb, sau putem să creem o componentă de 'configMap' care reprezintă o configurație externă, iar prin aceasta, alte componente au posibilitatea de o utiliza. De exemplu, dacă avem 2 aplicații care folosesc baza de date mongodb atunci se poate face referință doar la acel fișier extern de configurație, iar dacă trebuie modificată cale spre baza de date se va modifica doar într-un singur loc

<img src='./ScreenShots/Demo/mongo-express-deployment.png'>

Pentru moment fișierul de deployment o să arate precum este prezent în imaginea de mai sus. În continuare o să creem componenta de configmap. Această componentă se creează prin intermediul unui fișier de tip yaml.

<img src='./ScreenShots/Demo/mongo-configmap.png'>

Fișierul configMap este foarte asemănător cu cel de secret, avem un 'kind' care este de tipul 'ConfigMap', un nume trecut în partea de metadata și partea de 'data' unde avem trecute datele sub formatul key-value. (Secret, pe lângă acestea avea și partea de 'type: Opaque'). Pentru a referenția server-ul, trebuie doar să specificăm ca și value numele serviciului unde este prezentă baza de date. Acest nume este specificat în componenta de Service din mongodb (în cadrul fișierului 'mongobd-deployment.yaml') în secțiunea de metadata ('name: mongodb-service'). Prin urmare, referința va fi 'database_url: mongodb-service'

Precum partea de Secret, ordinea în care sunt create aceste componente contează, prin urmare trebuie să creem prima dată componenta de configmap pentru a putea face referință la aceasta în cadrul componentei de deployment.

In [ ]:
kubectl apply -f mongo-express-configmap.yaml

 <img src='./ScreenShots/Demo/kubectl_apply_mongo-express-configmap.png'>

Din moment ce avem componenta de configmap creată, putem să facem referire la aceasta în fișierul de deployment. Referința către aceasta se face foarte asemănător precum cea de Secret, diferența este că în loc de 'secretKeyRef' se va utiliza 'configMapKeyRef'

<img src='./ScreenShots/Demo/mongo-express-deployment2.png'>

Partea de deployment pentru mongo-express o să arate precum în imaginea de mai sus. După ce s-a creat și deployment-ul de mongo-express, pasul final al proiectului este să se afișeze mongo-express în cadrul unui browser. Pentru a putea face asta, avem nevoie de un service extern. Partea de service o să fie creată în cadrul fișierului 'mongo-express-deployment.yaml', precum am făcut și pentru imaginea de mongo.

Un service extern este foarte asemănător cu unul intern, singura diferență între acestea este că se va mai adăuga în partea de 'spec' încă un set de key-velue, iar partea de key este 'type'. Valoare pentru acest 'type' o să fie 'LoadBalancer'. Aceast LoadBalancer acceptă requesturi externe prin assignarea serviciului a unei adrese IP externe.

Următorul lucru care trebuie făcut în cadrul componentei de Service (un Service extern), în secțiunea de 'ports' mai trebuie specificat încă un port, iar acesta poartă denumirea de 'nodePort'. Acest 'nodePort' reprezintă portul extern pe care adresa IP externă o să îl deschidă. Acesta este port-ul care trebuie trecut în browser pentru a accesa serviciul. Valoare acestui port are un range, între 30000 și 32767

Aceste noi configurații o să creeze un service extern. Componenta finală de Service o să arate precum în imageinea de mai jos

<img src='./ScreenShots/Demo/mongo-express-deployment4.png'>

Dacă se afișează servicile prezente în clusterul de kubernetes, se poate observa că acest service nou ce l-am creat este de tipu-ul LoadBalancer, iar celelalte sunt de tipul 'ClusterIP'

<img src='./ScreenShots/Demo/kubectl_get_service.png'>

Diferența dintre cele 2 este că un 'ClusterIP' o să îi ofere service-ului o adresă IP internă, iar 'LoadBalancer' îi va da serviciului atât o adresă IP internă, dar și o adresă IP externă. În output scrie '<pending>' la adresa IP externă deoarece se utilizează minikube și funcționează puțin diferit. Într-un cluster kubernetes normal, acolo o să fie listată adresa IP externă.

În minikube, modul prin care putem face rost de adrsa IP externă a unui Service de tipul 'LoadBalancer' este prin utilizarea comenzii de 'minikube service'. Comanda aceasta o să îi assigneze unui service (care trebuie oferit ca și argument) o adresă IP externă. Comanda pentru a îi oferi service-ului extern de mongo-express o adresă IP externă este următoarea:

In [ ]:
minikube service mongo-express-service

În momentul în care se rulează comanda de mai sus, în browser o să se afișeze pagina de la mongo-express.

<img src='./ScreenShots/Demo/mongo-express-browser.png'>

În output-ul de pe terminal putem vedea informația cum că service-ului de mongo-express-service i-a fost atribuit o adresă IP externă, iar aceasta poate fi accesată de pe portul 30000, valoare pe care am oferit-o ca și key-value în cadrul componentei de Service în secțiunea de ports (nodePort: 30000)

Dacă se folosește broeser-ul pentru a crea o nouă bază de date, se va face un request către serverul extern (mongo-express-service) care o să trimită acest request către un pod de mongo-express. Pod-ul de mongo-express face conexiunea cu service-ul intern de mongodb (mongodb-service), iar acest service intern trimite mai departe request-ul către un pod de mongodb, iar în final baza de date este creată și pare în cadrul browser-ului. (Test-db este baza de date ce tocmai a fost creată).

<img src='./ScreenShots/Demo/mongo-express-browser2.png'>